In [ ]:
import torch
import torch.nn as nn
from conformer import Conformer
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

batch_size, sequence_length, dim = 3, 12345, 80

cuda = torch.cuda.is_available()  
device = torch.device('cuda' if cuda else 'cpu')

criterion = nn.CTCLoss().to(device)

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
processed_sample = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=16000)
processed_sample

In [2]:

inputs = torch.rand(batch_size, sequence_length, dim).to(device)
input_lengths = torch.LongTensor([12345, 12300, 12000])
targets = torch.LongTensor([[1, 3, 3, 3, 3, 3, 4, 5, 6, 2],
                            [1, 3, 3, 3, 3, 3, 4, 5, 2, 0],
                            [1, 3, 3, 3, 3, 3, 4, 2, 0, 0]]).to(device)
target_lengths = torch.LongTensor([9, 8, 7])



In [3]:
inputs.shape

torch.Size([3, 12345, 80])

In [4]:
input_lengths

tensor([12345, 12300, 12000])

In [5]:

model = Conformer(num_classes=10, 
                  input_dim=dim, 
                  encoder_dim=512, 
                  num_encoder_layers=3).to(device)

In [6]:
# Forward propagate
outputs, output_lengths = model(inputs, input_lengths)

# Calculate CTC Loss
loss = criterion(outputs.transpose(0, 1), targets, output_lengths, target_lengths)

In [7]:
outputs.shape

torch.Size([3, 1543, 10])

In [8]:
loss

tensor(415.3708, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
from datasets import load_dataset

ds = load_dataset("Aniemore/resd_annotated")